In [2]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../py_files')
from sdlnano import BayesianOptimizer_Auto
from calculation import BatchCalculation

In [ ]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [4]:
iteration = BayesianOptimizer_Auto.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 3


# Check the drug

In [5]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [6]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = BayesianOptimizer_Auto.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

[INFO 02-11 13:05:13] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 02-11 13:05:37] ax.service.ax_client: Generated new trial 48 with parameters {'Drug_MW': 0.354, 'Drug_LogP': 0.391, 'Drug_TPSA': 0.067, 'Drug': 1.0, 'SL_1': 0.0, 'SL_2': 0.0, 'SL_3': 0.086795, 'LL_1': 0.0, 'LL_2': 1.0, 'LL_3': 0.0, 'P_1': 0.180262, 'P_2': 0.174836, 'P_3': 0.763181, 'S_1': 0.0, 'S_2': 1.0, 'S_3': 0.819694, 'Water': 0.764441} using model BoTorch.
/Users/zeqing/opt/anaconda3/envs/sdlnano/lib/python3.10/site-packages/ax/core/data.py:288: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return cls(df=pd.con

7 min


In [ ]:
# process the trails into ratios

unlabeled_trial_processed = BayesianOptimizer_Auto.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.0,16.00,16.0,16.000000,16.00000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,55.500000,354.0,3.91,67.0,0.212063,0.01425,0.017250,0.039188,0.077813,0.228938,0.038250,0.078937,0.137688,0.155688,0.156250,0.389875,0.317063,0.136625
std,4.760952,0.0,0.00,0.0,0.096807,0.05700,0.059849,0.030922,0.121301,0.094778,0.082412,0.077949,0.110734,0.083762,0.352077,0.288020,0.242830,0.176671
min,48.000000,354.0,3.91,67.0,0.000000,0.00000,0.000000,0.009000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,51.750000,354.0,3.91,67.0,0.162750,0.00000,0.000000,0.016500,0.000000,0.197000,0.000000,0.021750,0.043750,0.108750,0.000000,0.258250,0.184750,0.000000
50%,55.500000,354.0,3.91,67.0,0.229000,0.00000,0.000000,0.029500,0.000000,0.238000,0.000000,0.058500,0.153000,0.157000,0.000000,0.360000,0.333000,0.000000
75%,59.250000,354.0,3.91,67.0,0.269250,0.00000,0.000000,0.053750,0.144000,0.289500,0.000000,0.115000,0.188750,0.222500,0.000000,0.571750,0.424750,0.333000
max,63.000000,354.0,3.91,67.0,0.356000,0.22800,0.239000,0.102000,0.388000,0.356000,0.221000,0.303000,0.381000,0.312000,1.000000,1.000000,0.966000,0.479000


In [8]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000,16.000,16.000,16.000000,16.0000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,55.500000,0.354,0.391,0.067,0.804777,0.0625,0.068835,0.139407,0.282797,0.858392,0.187500,0.301444,0.502616,0.578571,0.187500,0.752217,0.641116,0.370633
std,4.760952,0.000,0.000,0.000,0.342232,0.2500,0.249595,0.112239,0.420659,0.319912,0.403113,0.283513,0.363625,0.301526,0.403113,0.443318,0.413459,0.477069
min,48.000000,0.354,0.391,0.067,0.000000,0.0000,0.000000,0.047181,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,51.750000,0.354,0.391,0.067,0.716067,0.0000,0.000000,0.055781,0.000000,1.000000,0.000000,0.082194,0.142036,0.344205,0.000000,0.758869,0.249189,0.000000
50%,55.500000,0.354,0.391,0.067,1.000000,0.0000,0.000000,0.110590,0.000000,1.000000,0.000000,0.289626,0.577273,0.678824,0.000000,1.000000,0.787063,0.000000
75%,59.250000,0.354,0.391,0.067,1.000000,0.0000,0.000000,0.172158,0.625425,1.000000,0.000000,0.426463,0.804417,0.834527,0.000000,1.000000,1.000000,1.000000
max,63.000000,0.354,0.391,0.067,1.000000,1.0000,1.000000,0.447523,1.000000,1.000000,1.000000,1.000000,1.000000,0.932898,1.000000,1.000000,1.000000,1.000000


In [ ]:
# Convert ratios to volumes

transfer = BatchCalculation.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.0,16.00,16.0,16.000000,16.0000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.00000,16.000000,16.000000,16.000000,16.000000
mean,55.500000,354.0,3.91,67.0,74.221875,4.9875,6.037500,13.715625,27.234375,80.128125,13.387500,27.628125,48.190625,54.490625,156.25000,389.875000,317.062500,136.625000,-0.021875
std,4.760952,0.0,0.00,0.0,33.882331,19.9500,20.947239,10.822657,42.455489,33.172287,28.844332,27.282186,38.756811,29.316748,352.07717,288.020341,242.829836,176.671399,0.298869
min,48.000000,354.0,3.91,67.0,0.000000,0.0000,0.000000,3.150000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,-0.700000
25%,51.750000,354.0,3.91,67.0,56.962500,0.0000,0.000000,5.775000,0.000000,68.950000,0.000000,7.612500,15.312500,38.062500,0.00000,258.250000,184.750000,0.000000,-0.087500
50%,55.500000,354.0,3.91,67.0,80.150000,0.0000,0.000000,10.325000,0.000000,83.300000,0.000000,20.475000,53.550000,54.950000,0.00000,360.000000,333.000000,0.000000,0.000000
75%,59.250000,354.0,3.91,67.0,94.237500,0.0000,0.000000,18.812500,50.400000,101.325000,0.000000,40.250000,66.062500,77.875000,0.00000,571.750000,424.750000,333.000000,0.087500
max,63.000000,354.0,3.91,67.0,124.600000,79.8000,83.650000,35.700000,135.800000,124.600000,77.350000,106.050000,133.350000,109.200000,1000.00000,1000.000000,966.000000,479.000000,0.350000


In [10]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug       1.2
SL_1       0.1
SL_2       0.1
SL_3       0.2
LL_1       0.4
LL_2       1.3
LL_3       0.2
P_1        0.4
P_2        0.8
P_3        0.9
S_1        2.5
S_2        6.2
S_3        5.1
Water      2.2
Solvent   -0.0
dtype: float64

In [11]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [12]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.drop(['Drug_MW', 'Drug_LogP', 'Drug_TPSA'], axis=1).to_excel(transfer_path, index = False)

[INFO 02-11 13:12:52] ax.service.ax_client: Saved JSON-serialized state of optimization to `optimizer/optimizer_design_3.json`.


# Generate new protocol

In [13]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_.py'


In [14]:
# Update the script with Excel data
updated_script = BatchCalculation.update_transfer_script(template_protocol, transfer_path, output_script_path) 


In [15]:
updated_script

'protocol/iteration_3_OT_2_protocol_.py'

# Perform experiment

# Results

In [16]:
results = BatchCalculation.parameters_and_complexity(iteration)
results

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,48,354,3.91,67,0.312,0.000,0.000,0.027,0.000,0.312,0.000,0.056,0.055,0.238,0.0,0.387,0.317,0.296,7,0
1,49,354,3.91,67,0.144,0.000,0.037,0.065,0.071,0.197,0.000,0.118,0.266,0.102,0.0,0.476,0.445,0.079,9,0
2,50,354,3.91,67,0.264,0.000,0.000,0.015,0.000,0.264,0.000,0.029,0.209,0.219,0.0,0.768,0.232,0.000,7,0
3,51,354,3.91,67,0.239,0.000,0.239,0.032,0.000,0.239,0.000,0.095,0.010,0.147,0.0,0.333,0.333,0.333,8,0
4,52,354,3.91,67,0.194,0.000,0.000,0.010,0.000,0.194,0.194,0.061,0.182,0.167,0.0,1.000,0.000,0.000,7,0
5,53,354,3.91,67,0.221,0.000,0.000,0.033,0.204,0.221,0.000,0.114,0.090,0.118,0.0,0.577,0.423,0.000,8,0
6,54,354,3.91,67,0.335,0.000,0.000,0.018,0.000,0.335,0.000,0.000,0.000,0.312,0.0,0.333,0.333,0.333,5,0
7,55,354,3.91,67,0.197,0.000,0.000,0.009,0.000,0.197,0.197,0.053,0.178,0.168,1.0,0.000,0.000,0.000,7,0
8,56,354,3.91,67,0.049,0.000,0.000,0.075,0.388,0.000,0.000,0.050,0.381,0.058,0.0,0.333,0.333,0.333,7,0
9,57,354,3.91,67,0.251,0.000,0.000,0.013,0.000,0.251,0.000,0.000,0.251,0.233,0.0,0.479,0.043,0.479,6,0


In [17]:
size_raw = BatchCalculation.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Good,A1,257.636338,0.244003
1,Good,A2,267.088851,0.205482
2,Good,A3,272.423473,0.255781
3,Good,A4,270.991893,0.268521
4,Good,A5,261.417151,0.260011
5,Good,A6,244.104668,0.369161
6,Good,B1,232.729507,0.223210
7,Good,B2,224.852850,0.163807
8,Good,B3,224.542129,0.206828
9,Good,B4,302.822126,0.261574


In [18]:
size_processed = BatchCalculation.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,265.716221,7.488520,0.235088,0.026308,1
1,Formulation 2,258.837904,13.627916,0.299231,0.060711,1
2,Formulation 3,227.374829,4.639889,0.197948,0.030681,1
3,Formulation 4,303.613105,1.118613,0.265757,0.005916,1
4,Formulation 5,235.303348,6.421513,0.237915,0.043763,1
5,Formulation 6,222.883115,3.510000,0.209439,0.046425,1
6,Formulation 7,246.761715,8.664952,0.224112,0.033136,1
7,Formulation 8,242.386166,1.269570,0.405178,0.072019,1
8,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0
9,Formulation 10,208.426089,2.515758,0.227249,0.012112,1


In [19]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,48,354,3.91,67,0.312,0.000,0.000,0.027,0.000,0.312,...,0.317,0.296,7,0,Formulation 1,265.716221,7.488520,0.235088,0.026308,1
1,49,354,3.91,67,0.144,0.000,0.037,0.065,0.071,0.197,...,0.445,0.079,9,0,Formulation 2,258.837904,13.627916,0.299231,0.060711,1
2,50,354,3.91,67,0.264,0.000,0.000,0.015,0.000,0.264,...,0.232,0.000,7,0,Formulation 3,227.374829,4.639889,0.197948,0.030681,1
3,51,354,3.91,67,0.239,0.000,0.239,0.032,0.000,0.239,...,0.333,0.333,8,0,Formulation 4,303.613105,1.118613,0.265757,0.005916,1
4,52,354,3.91,67,0.194,0.000,0.000,0.010,0.000,0.194,...,0.000,0.000,7,0,Formulation 5,235.303348,6.421513,0.237915,0.043763,1
5,53,354,3.91,67,0.221,0.000,0.000,0.033,0.204,0.221,...,0.423,0.000,8,0,Formulation 6,222.883115,3.510000,0.209439,0.046425,1
6,54,354,3.91,67,0.335,0.000,0.000,0.018,0.000,0.335,...,0.333,0.333,5,0,Formulation 7,246.761715,8.664952,0.224112,0.033136,1
7,55,354,3.91,67,0.197,0.000,0.000,0.009,0.000,0.197,...,0.000,0.000,7,0,Formulation 8,242.386166,1.269570,0.405178,0.072019,1
8,56,354,3.91,67,0.049,0.000,0.000,0.075,0.388,0.000,...,0.333,0.333,7,0,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0
9,57,354,3.91,67,0.251,0.000,0.000,0.013,0.000,0.251,...,0.043,0.479,6,0,Formulation 10,208.426089,2.515758,0.227249,0.012112,1


In [20]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,48,354,3.91,67,0.312,0.000,0.000,0.027,0.000,0.312,...,7,0,Formulation 1,265.716221,7.488520,0.235088,0.026308,1,624.0,0
1,49,354,3.91,67,0.144,0.000,0.037,0.065,0.071,0.197,...,9,0,Formulation 2,258.837904,13.627916,0.299231,0.060711,1,288.0,0
2,50,354,3.91,67,0.264,0.000,0.000,0.015,0.000,0.264,...,7,0,Formulation 3,227.374829,4.639889,0.197948,0.030681,1,528.0,0
3,51,354,3.91,67,0.239,0.000,0.239,0.032,0.000,0.239,...,8,0,Formulation 4,303.613105,1.118613,0.265757,0.005916,1,478.0,0
4,52,354,3.91,67,0.194,0.000,0.000,0.010,0.000,0.194,...,7,0,Formulation 5,235.303348,6.421513,0.237915,0.043763,1,388.0,0
5,53,354,3.91,67,0.221,0.000,0.000,0.033,0.204,0.221,...,8,0,Formulation 6,222.883115,3.510000,0.209439,0.046425,1,442.0,0
6,54,354,3.91,67,0.335,0.000,0.000,0.018,0.000,0.335,...,5,0,Formulation 7,246.761715,8.664952,0.224112,0.033136,1,670.0,0
7,55,354,3.91,67,0.197,0.000,0.000,0.009,0.000,0.197,...,7,0,Formulation 8,242.386166,1.269570,0.405178,0.072019,1,394.0,0
8,56,354,3.91,67,0.049,0.000,0.000,0.075,0.388,0.000,...,7,0,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
9,57,354,3.91,67,0.251,0.000,0.000,0.013,0.000,0.251,...,6,0,Formulation 10,208.426089,2.515758,0.227249,0.012112,1,502.0,0


In [21]:
results_normalized = BayesianOptimizer_Auto.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.265716,0.007489,0.312,0.0,0.235088,0.026308,0.583333
1,1,0.258838,0.013628,0.144,0.0,0.299231,0.060711,0.750000
2,1,0.227375,0.004640,0.264,0.0,0.197948,0.030681,0.583333
3,1,0.303613,0.001119,0.239,0.0,0.265757,0.005916,0.666667
4,1,0.235303,0.006422,0.194,0.0,0.237915,0.043763,0.583333
5,1,0.222883,0.003510,0.221,0.0,0.209439,0.046425,0.666667
6,1,0.246762,0.008665,0.335,0.0,0.224112,0.033136,0.416667
7,1,0.242386,0.001270,0.197,0.0,0.405178,0.072019,0.583333
8,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.583333
9,1,0.208426,0.002516,0.251,0.0,0.227249,0.012112,0.500000


In [22]:
results_normalized = BayesianOptimizer_Auto.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.265716,0.007489,0.312,0.0,0.235088,0.026308,0.583333
1,1,0.258838,0.013628,0.144,0.0,0.299231,0.060711,0.750000
2,1,0.227375,0.004640,0.264,0.0,0.197948,0.030681,0.583333
3,1,0.303613,0.001119,0.239,0.0,0.265757,0.005916,0.666667
4,1,0.235303,0.006422,0.194,0.0,0.237915,0.043763,0.583333
5,1,0.222883,0.003510,0.221,0.0,0.209439,0.046425,0.666667
6,1,0.246762,0.008665,0.335,0.0,0.224112,0.033136,0.416667
7,1,0.242386,0.001270,0.197,0.0,0.405178,0.072019,0.583333
8,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.583333
9,1,0.208426,0.002516,0.251,0.0,0.227249,0.012112,0.500000


In [23]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [24]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [25]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,48,354,3.91,67,0.312,0.0,0.000,0.027,0.000,0.312,...,0.583333,0,Formulation 1,0.265716,0.007489,0.235088,0.026308,1,0.312,0
1,49,354,3.91,67,0.144,0.0,0.037,0.065,0.071,0.197,...,0.750000,0,Formulation 2,0.258838,0.013628,0.299231,0.060711,1,0.144,0
2,50,354,3.91,67,0.264,0.0,0.000,0.015,0.000,0.264,...,0.583333,0,Formulation 3,0.227375,0.004640,0.197948,0.030681,1,0.264,0
3,51,354,3.91,67,0.239,0.0,0.239,0.032,0.000,0.239,...,0.666667,0,Formulation 4,0.303613,0.001119,0.265757,0.005916,1,0.239,0
4,52,354,3.91,67,0.194,0.0,0.000,0.010,0.000,0.194,...,0.583333,0,Formulation 5,0.235303,0.006422,0.237915,0.043763,1,0.194,0


In [26]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


[INFO 02-11 15:10:55] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.


In [27]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [28]:
loaded_ax_client = BayesianOptimizer_Auto.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 02-11 15:10:55] ax.service.ax_client: Completed trial 48 with data: {'Solu': (0.312, 0), 'Size': (0.265716, 0.007489), 'PDI': (0.235088, 0.026308), 'Complexity': (0.583333, 0)}.
[INFO 02-11 15:10:55] ax.service.ax_client: Completed trial 49 with data: {'Solu': (0.144, 0), 'Size': (0.258838, 0.013628), 'PDI': (0.299231, 0.060711), 'Complexity': (0.75, 0)}.
[INFO 02-11 15:10:55] ax.service.ax_client: Completed trial 50 with data: {'Solu': (0.264, 0), 'Size': (0.227375, 0.00464), 'PDI': (0.197948, 0.030681), 'Complexity': (0.583333, 0)}.
[INFO 02-11 15:10:55] ax.service.ax_client: Completed trial 51 with data: {'Solu': (0.239, 0), 'Size': (0.303613, 0.001119), 'PDI': (0.265757, 0.005916), 'Complexity': (0.666667, 0)}.
[INFO 02-11 15:10:55] ax.service.ax_client: Completed trial 52 with data: {'Solu': (0.194, 0), 'Size': (0.235303, 0.006422), 'PDI': (0.237915, 0.043763), 'Complexity': (0.583333, 0)}.
[INFO 02-11 15:10:56] ax.service.ax_client: Completed trial 53 with data: {'Solu': (0